https://hd.stheadline.com/news/realtime/fin/2391104/%E5%8D%B3%E6%99%82-%E9%87%91%E8%9E%8D-%E6%B8%A3%E6%89%93%E4%BF%A1%E7%94%A8%E5%8D%A1%E7%88%86%E5%A4%A7%E8%A6%8F%E6%A8%A1%E7%9B%9C%E7%94%A8-%E5%AE%A2%E6%9C%8D%E5%94%94%E9%80%9A-%E6%89%8B%E6%A9%9F%E6%AD%BBApp-%E6%B8%A3%E6%89%93%E6%99%9A%E4%B8%8A%E6%94%B9%E5%AE%89%E6%8E%92-%E5%B0%87%E4%B8%BB%E5%8B%95%E9%80%80%E6%AC%BE-%E7%84%A1%E8%AD%89%E6%93%9A%E8%B3%87%E6%96%99%E5%A4%96%E6%B4%A9

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import sys
import time
import requests
from itertools import cycle
from datetime import datetime
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchWindowException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
import undetected_chromedriver as uc
from datetime import timedelta, date, datetime
from dateutil.relativedelta import relativedelta

In [2]:
#functions
def click_thread(idx):
    #click thread
    auto_xpath = f'//*[@id="leftPanel"]/div[2]/div[{idx + 1}]/div/div[2]/a[1]'
    try:
        driver.find_element(By.XPATH, auto_xpath).click()
        #print(idx)
    except:
        #print("skip")
        pass
    time.sleep(0.3)
    
def scroll_left_panel():
    counter = 0
    while True:
        html = driver.page_source
        soup = BeautifulSoup(html)
        checker = soup.find_all("div",{"class":"_21IQKhlBjN2jlHS_TVgI3l"})
        
        counter = counter + 1 
        previous_checker = len(checker)
        if counter == 10:
            try:
                if len(checker) == previous_checker:
                    break
            except:
                pass

        try:
            driver.find_element(By.CLASS_NAME, '_33r1FGqGJZF-fM1VZm7mhN').text
            time.sleep(2)
            break
        except:
            pass

        for i in range(20):
            ActionChains(driver)\
                .send_keys(" ")\
                .perform()
            time.sleep(0.1)

In [3]:
#Open pandas
try:
    df_thread_log = pd.read_excel('LIHKG_Thread_Log.xlsx')
except:
    df_thread_log = pd.DataFrame(columns=["Thread UID (PK)",
                                       "Keyword",
                                       "Reference Bank",
                                       "Thread Post Datetime",
                                       "Thread Author",
                                       "Thread Title",
                                       "Thread Theme",                                    
                                       "Thread Likes",
                                       "Thread Dislike",
                                       "Last Fetch Comment Floor Number"])
    
try:
    df_fetch_log = pd.read_excel('LIHKG_Fetch_Log.xlsx')
except:
    df_fetch_log = pd.DataFrame(columns=["Fetch UID (PK)",
                                       "Thread UID (FK)",
                                       "Fetch Datetime",
                                       "Thread Lastest Comment Time(Thread info)",
                                       "Thread Lastest Comment Time(Fetch inside comment)",
                                       "Thread Number of Page"])

#Get Distinct value by last row

try:
    df_distinct_thread_by_latest = df_thread_log.drop_duplicates(subset=['Keyword',
                                                                          'Thread Author',
                                                                          'Thread Title',
                                                                          'Thread Theme'], keep='last')
except:
    pass
    
try:
    df_comments = pd.read_excel('LIHKG_Comment_Level.xlsx')
except:
    df_comments = pd.DataFrame(columns=["Thread UID",
                                        "Comment UID",
                                        "Fetch Datetime",
                                        "Comment Floor Number",
                                        "Comment Author",
                                        "Comment Datetime",
                                        "Comment Date",
                                        "Comment Time",
                                        "Comment Likes",
                                        "Comment Dislike",
                                        "Comment Replies",
                                        "Comment Context"])   

df_keyword = pd.read_excel('LIHKG_Keyword.xlsx', index_col=None)
keyword_list = df_keyword["Keyword"].values.tolist()
ref_bank_list = df_keyword["Reference Bank"].values.tolist()
df_keyword

,Keyword,Reference Bank
0,渣打,Standard Chartered
1,揸兜,Standard Chartered
2,渣打銀行,Standard Chartered


In [4]:
%%time
#set driver to uc driver
driver = uc.Chrome()

########
# MAIN #
########

#starts of the loop

for keyword_row_num, keyword in enumerate(keyword_list):
    ref_bank = ref_bank_list[keyword_row_num]
    fetch_datetime = datetime.now().strftime("%Y%m%d-%H%M%S%f")
    #keyword_uid = "LI-"+str(fetch_datetime)+"-"+ str(keyword_row_num).zfill(8)
    fetch_uid = "LI-"+ str(fetch_datetime)

    #get link
    driver.get(f"https://lihkg.com/search?q={keyword}&sort=desc_create_time&type=thread")
    time.sleep(4)
    
    #get html info
    html = driver.page_source
    soup = BeautifulSoup(html)
    
    #click left panel for scrolling
    driver.find_element(By.XPATH, '//*[@id="leftPanel"]/div[1]/ul/li[2]/a').click()

    #Call the function and scroll to the end (leftpanel)
    scroll_left_panel()
    
    #get latest html info
    html = driver.page_source
    soup = BeautifulSoup(html)
    
    #get thread html info
    threads = soup.find("div",{"class":"qoAmEqNpZRLf2KVKZ8DsC"})
    for thread_idx, i in enumerate(threads):
        #click thread
        click_thread(thread_idx)
        #give buffer incase the webpage use too much resource.
        time.sleep(0.2)
        
        #get thread info
        try:
            try:
                thread_author = i.find("span", {"class":"CxY4XDSSItTeLVg0cKCN0 A0jheqYUBHNW93KykXKEH"}).text
            except:
                thread_author = i.find("span", {"class":"CxY4XDSSItTeLVg0cKCN0 jj_3ZDzjtPixL1b2KTcpS"}).text

            thread_lastest_comment_time = i.find("span", {"class":"_37XwjAqVHtjzqzEtybpHrU"}).text
            thread_lastest_comment_time.replace('年',"-").replace('月', "-").replace('日', "")
            
            ###
            #check day or hr or month function & break it when the time is not in range
            #
            #
            ###

            thread_title = i.find("span", {"class":"_20jopXBFHNQ9FUbcGHLcHH"}).text
            thread_page = i.find("div", {"class":"_26oEXjfUS_iHzbxYcZE6bD"}).text.split(" ")[0]
            thread_theme = i.find_all("a")[1].text
            thread_total_like = driver.find_elements(By.CLASS_NAME, '_8_NT40G-QNQzcSSTrRXAD')[0].get_attribute('data-score')
            thread_total_dislike = driver.find_elements(By.CLASS_NAME, '_8_NT40G-QNQzcSSTrRXAD')[1].get_attribute('data-score')

        except:
            pass
        
        #Locate the thread in excel
        try:
            floor_number_loc = df_distinct_thread_by_latest.loc[(df_distinct_thread_by_latest['Keyword'] == keyword)
                                                                & (df_distinct_thread_by_latest['Thread Author'] == thread_author) 
                                                                & (df_distinct_thread_by_latest['Thread Title'] == thread_title)
                                                                & (df_distinct_thread_by_latest['Thread Theme'] == thread_theme)]
            floor_number_pointer = floor_number_loc["Last Fetch Comment Floor Number"].values[0]
        except:
            floor_number_pointer = '#0'
            
        #current time
        fetch_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

        #get the latest page info
        html = driver.page_source
        soup = BeautifulSoup(html)

        #loop to the end of pages
        for click in range(int(thread_page)):
            '''
            ###
            very rare case when occur a thread that have very longgggg
            context in the first post, the f5 detect will not working.
            Thus it will skip the scrolling and fetch data directly.
            ###
            '''
            try:
                find_f5 = driver.find_element(By.CLASS_NAME, '_1PhR8JHkMcET5QT2PtCA3T')
                actions = ActionChains(driver)
                actions.move_to_element(find_f5).perform()
                time.sleep(1)
            except:
                try:
                    find_f5 = driver.find_element(By.XPATH, f'//*[@id="page-{str(click+1)}"]/div[3]/a').click()
                except:
                    for j in range(20):
                        ActionChains(driver).send_keys(" ").perform()

                        
        #get the latest page info
        html = driver.page_source
        soup = BeautifulSoup(html)
        
        #date checker, to stop the current loop
        date_out_of_range = False
        
        #start of fetching comments data
        all_comment = soup.find_all("div",{"class":"_36ZEkSvpdj_igmog0nluzh"})
        for comment_idx , comment in enumerate(all_comment):
            
            comment_floor_number = comment.find_all("span")[0].text
            comment_author = comment.find("span",{"class":"ZZtOrmcIRcvdpnW09DzFk"}).text

            #incase the comment is folded due to not enough info
            try:
                comment_datetime = comment.find("span",{"class":"Ahi80YgykKo22njTSCzs_"})
                new_comment_datetime = str(comment_datetime).split("\"")[5]
                new_comment_datetime = new_comment_datetime.replace('年',"-").replace('月', "-").replace('日', "")
                comment_date = new_comment_datetime.split(" ")[0]
                comment_time = new_comment_datetime.split(" ")[1]

                comment_like = comment.find_all("label")[1].text
                comment_dislike = comment.find_all("label")[3].text
            except:
                comment_datetime = ""
                comment_date = ''
                comment_time = ''
                comment_like = 0
                comment_dislike = 0

            try:
                comment_replies = comment.find_all("label")[5].text
            except:
                comment_replies = 0

            try:
                comment_context = comment.find("div",{"class":"_2cNsJna0_hV8tdMj3X6_gJ"}).text
            except:
                comment_context = ''
            
            #find earliest time
            if comment_idx == 0:
                #check the datetime is within 3 month or not
                thread_time = datetime.strptime(comment_date, '%Y-%m-%d').date()
                cur_time = datetime.now().strftime("%Y-%m-%d")
                cur_time = datetime.strptime(cur_time, '%Y-%m-%d').date()
                
                if thread_time < cur_time - relativedelta(months=3):
                    date_out_of_range = True
                    print(cur_time,thread_time)
                    break
    
                thread_post_datetime = new_comment_datetime
                ######
                #create thread_uid by reverse order
                thread_uid = fetch_uid + "-" + thread_author + "-" + thread_post_datetime
                ######
                
            comment_uid = thread_uid + "-" + str(comment_floor_number.replace("#", "")).zfill(8)

            #get the lastest comment time
            latest_comment_time = new_comment_datetime
                
            #get lastest floor number in this fetch
            lastest_floor_number = comment_floor_number
            
            #current time
            fetch_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            
            #compare the latest floor number
            if int(floor_number_pointer.replace("#", "")) < int(comment_floor_number.replace("#", "")):
                df_comments.loc[len(df_comments)]= [thread_uid,
                                                     comment_uid,
                                                     fetch_datetime,
                                                     comment_floor_number,
                                                     comment_author,
                                                     new_comment_datetime,
                                                     comment_date,
                                                     comment_time,
                                                     int(comment_like),
                                                     int(comment_dislike), 
                                                     int(comment_replies), 
                                                     comment_context]
                
        #break the current threads loop if the date is larger than 3 month
        if date_out_of_range == True:
            break

        
        #save thread into to dataframe
        df_thread_log.loc[len(df_thread_log)]= [thread_uid,
                                          keyword,
                                          ref_bank,                                         
                                          thread_post_datetime,
                                          thread_author, 
                                          thread_title,
                                          thread_theme, 
                                          int(thread_total_like), 
                                          int(thread_total_dislike),
                                          lastest_floor_number]

        df_fetch_log.loc[len(df_fetch_log)]= [fetch_uid,
                                          thread_uid,
                                          fetch_datetime,                                          
                                          thread_lastest_comment_time,
                                          latest_comment_time,
                                          int(thread_page)]
'''
        if thread_idx == 1:
            break
    if keyword_row_num == 0:
        break
'''
driver.quit()

2023-01-10 2022-10-08
2023-01-10 2022-09-17
2023-01-10 2022-08-18
Wall time: 12min 21s


In [5]:
df_thread_log

,Thread UID (PK),Keyword,Reference Bank,Thread Post Datetime,Thread Author,Thread Title,Thread Theme,Thread Likes,Thread Dislike,Last Fetch Comment Floor Number
0,LI-20230110-232042066087-瀧野由美仔-2023-1-5 22:38:13,渣打,Standard Chartered,2023-1-5 22:38:13,瀧野由美仔,viu邊度黃呀？mirror邊度黃呀？反tvb就叫黃？鬼打鬼渣喎，你咪又係撐緊藍？,娛樂台,9,7,#11
1,LI-20230110-232042066087-格林斯潘-2023-1-5 22:19:37,渣打,Standard Chartered,2023-1-5 22:19:37,格林斯潘,中東阿布扎比銀行考慮收購渣打,財經台,5,1,#13
2,LI-20230110-232042066087-只嫁紀律部隊-2023-1-5 14:48:32,渣打,Standard Chartered,2023-1-5 14:48:32,只嫁紀律部隊,渣打定期5.25,財經台,0,3,#9
3,LI-20230110-232042066087-3noirvexa老師-2023-1-5 ...,渣打,Standard Chartered,2023-1-5 12:27:48,3noirvexa老師,邊間渣打多靚女推介,體育台,0,1,#5
4,LI-20230110-232042066087-王頌茵-2023-1-3 05:46:42,渣打,Standard Chartered,2023-1-3 05:46:42,王頌茵,渣打比人盜用左條數係唔係要找??,財經台,0,6,#27
...,...,...,...,...,...,...,...,...,...,...
68,LI-20230110-232907058447-囝細老婆嫩-2022-12-7 22:45:04,揸兜,Standard Chartered,2022-12-7 22:45:04,囝細老婆嫩,睇嚟揸兜今日好唔掂喎⋯⋯,吹水台,2,0,#4
69,LI-20230110-232907058447-神宮寺-2022-12-6 21:04:25,揸兜,Standard Chartered,2022-12-6 21:04:25,神宮寺,揸兜信用卡比人碌到錢,財經台,33,465,#1001
70,LI-20230110-232907058447-納米屌洗衣液-2022-10-11 11:...,揸兜,Standard Chartered,2022-10-11 11:59:24,納米屌洗衣液,mpf咁撚樣搞 係咪以後65歲後個個揸兜？,財經台,1055,11,#241
71,LI-20230110-233246137217-格林斯潘-2023-1-5 22:19:37,渣打銀行,Standard Chartered,2023-1-5 22:19:37,格林斯潘,中東阿布扎比銀行考慮收購渣打,財經台,5,1,#13


In [6]:
df_fetch_log

,Fetch UID (PK),Thread UID (FK),Fetch Datetime,Thread Lastest Comment Time(Thread info),Thread Lastest Comment Time(Fetch inside comment),Thread Number of Page
0,LI-20230110-232042066087,LI-20230110-232042066087-瀧野由美仔-2023-1-5 22:38:13,2023-01-10 23:21:08,2 日前,2023-1-6 15:24:17,1
1,LI-20230110-232042066087,LI-20230110-232042066087-格林斯潘-2023-1-5 22:19:37,2023-01-10 23:21:11,4 日前,2023-1-5 23:33:26,1
2,LI-20230110-232042066087,LI-20230110-232042066087-只嫁紀律部隊-2023-1-5 14:48:32,2023-01-10 23:21:14,4 日前,2023-1-6 15:19:36,1
3,LI-20230110-232042066087,LI-20230110-232042066087-3noirvexa老師-2023-1-5 ...,2023-01-10 23:21:16,5 日前,2023-1-5 15:02:15,1
4,LI-20230110-232042066087,LI-20230110-232042066087-王頌茵-2023-1-3 05:46:42,2023-01-10 23:21:21,4 日前,2023-1-6 15:29:37,2
...,...,...,...,...,...,...
68,LI-20230110-232907058447,LI-20230110-232907058447-囝細老婆嫩-2022-12-7 22:45:04,2023-01-10 23:29:33,1 個月前,2022-12-7 23:15:47,1
69,LI-20230110-232907058447,LI-20230110-232907058447-神宮寺-2022-12-6 21:04:25,2023-01-10 23:32:20,1 個月前,2022-12-7 15:17:16,41
70,LI-20230110-232907058447,LI-20230110-232907058447-納米屌洗衣液-2022-10-11 11:...,2023-01-10 23:32:42,2 個月前,2022-10-13 23:09:39,10
71,LI-20230110-233246137217,LI-20230110-233246137217-格林斯潘-2023-1-5 22:19:37,2023-01-10 23:32:57,4 日前,2023-1-5 23:33:26,1


In [7]:
df_comments

,Thread UID,Comment UID,Fetch Datetime,Comment Floor Number,Comment Author,Comment Datetime,Comment Date,Comment Time,Comment Likes,Comment Dislike,Comment Replies,Comment Context
0,LI-20230110-232042066087-瀧野由美仔-2023-1-5 22:38:13,LI-20230110-232042066087-瀧野由美仔-2023-1-5 22:38:...,2023-01-10 23:21:08,#1,瀧野由美仔,2023-1-5 22:38:13,2023-1-5,22:38:13,9,7,0,仲要呢隻叫鬼鼠藍，披著黃皮既藍！藍撚到黑！
1,LI-20230110-232042066087-瀧野由美仔-2023-1-5 22:38:13,LI-20230110-232042066087-瀧野由美仔-2023-1-5 22:38:...,2023-01-10 23:21:08,#2,BenWhite,2023-1-5 22:38:45,2023-1-5,22:38:45,6,1,1,黃尸依家先知但又返唔到轉頭
2,LI-20230110-232042066087-瀧野由美仔-2023-1-5 22:38:13,LI-20230110-232042066087-瀧野由美仔-2023-1-5 22:38:...,2023-01-10 23:21:08,#3,LittleBraver,2023-1-5 22:39:33,2023-1-5,22:39:33,8,0,0,黃唔黃唔知 但我淨係知mirror係垃圾\n不服都唔好來辯
3,LI-20230110-232042066087-瀧野由美仔-2023-1-5 22:38:13,LI-20230110-232042066087-瀧野由美仔-2023-1-5 22:38:...,2023-01-10 23:21:08,#4,TVBPR,2023-1-5 22:40:11,2023-1-5,22:40:11,0,0,0,鄭州加油
4,LI-20230110-232042066087-瀧野由美仔-2023-1-5 22:38:13,LI-20230110-232042066087-瀧野由美仔-2023-1-5 22:38:...,2023-01-10 23:21:08,#5,大碌鹿班比,2023-1-5 22:41:03,2023-1-5,22:41:03,2,0,0,Viu台之前冇乜fans咪扮下野\n\n依家搵到fans base咪露出真面目囉\n\n以前...
...,...,...,...,...,...,...,...,...,...,...,...,...
5169,LI-20230110-233246137217-馬里蘭樂園田泰安-2022-12-29 2...,LI-20230110-233246137217-馬里蘭樂園田泰安-2022-12-29 2...,2023-01-10 23:32:58,#6,一個人二張櫈,2022-12-29 20:33:26,2022-12-29,20:33:26,1,0,0,反而iphone 下面果一劃成日影響我㩒最底果個位嘅制佢個畫面又唔可以再拉高啲\n\n有冇方...
5170,LI-20230110-233246137217-馬里蘭樂園田泰安-2022-12-29 2...,LI-20230110-233246137217-馬里蘭樂園田泰安-2022-12-29 2...,2023-01-10 23:32:58,#7,瀟灑不J,2022-12-29 20:42:20,2022-12-29,20:42:20,4,0,0,真，屌佢老母IT狗，點解唔set 死右上角或下面要整浮動icon\n拉黎拉去都麻鳩煩啦\n問...
5171,LI-20230110-233246137217-馬里蘭樂園田泰安-2022-12-29 2...,LI-20230110-233246137217-馬里蘭樂園田泰安-2022-12-29 2...,2023-01-10 23:32:58,#8,輸入用戶名稱,2022-12-29 20:44:19,2022-12-29,20:44:19,1,0,0,渣打app一向弱智
5172,LI-20230110-233246137217-馬里蘭樂園田泰安-2022-12-29 2...,LI-20230110-233246137217-馬里蘭樂園田泰安-2022-12-29 2...,2023-01-10 23:32:58,#9,博５刀,2022-12-29 21:08:22,2022-12-29,21:08:22,1,0,0,都on9啦 成日要拉上拉落


In [8]:
#output data
df_thread_log.to_excel('LIHKG_Thread_Log.xlsx',encoding= 'utf-8-sig',index = None) 
df_fetch_log.to_excel('LIHKG_Fetch_Log.xlsx',encoding= 'utf-8-sig',index = None) 
df_comments.to_excel('LIHKG_Comment_Level.xlsx',encoding= 'utf-8-sig',index = None) 

C:\Users\User\anaconda3\lib\site-packages\pandas\util\_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)


In [9]:
#read and save it without null, need to find another way be smart
df_comments = pd.read_excel('LIHKG_Comment_Level.xlsx')
#drop null value
df_comments = df_comments.dropna()
df_comments.to_excel('LIHKG_Comment_Level.xlsx',encoding= 'utf-8-sig',index = None) 

C:\Users\User\anaconda3\lib\site-packages\pandas\util\_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)
